In [1]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

import time

In [2]:
country = "AR"
nq_date = "190912"

In [3]:
census = pd.read_csv("../Data/"+country+"/sample/"+"ipumsi_00015.csv")

In [4]:
netquest = pd.read_csv("../Data/"+country+"/panel/"+country+"_netquest-panel.csv")

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
nq_dict = pd.read_excel("../Data/AR/AR_levels.xlsx",
                       )

In [6]:
geo1_nq = "AR_provincia"
geo2_nq = "AR_departamento"
geo1_ipums = 'GEO1_AR2010'
geo2_ipums = 'GEO2_AR2010'
year = '2010'

In [7]:
ipums_geo2 = pd.read_csv('../Data/'+country+'/sample/ipums_codebook_'+geo2_ipums+'.csv',encoding='Latin1', names=['code','name'], skiprows=1)
ipums_geo1 = pd.read_csv('../Data/'+country+'/sample/ipums_codebook_'+geo1_ipums+'.csv',encoding='Latin1', names=['code','name'], skiprows=1)

In [8]:
nq_geo1 = nq_dict[nq_dict.Variable==geo1_nq]
nq_geo1.columns = ["Variable",geo1_nq,geo1_nq+"_name"]
nq_geo1 = nq_geo1[[geo1_nq,geo1_nq+"_name"]]

nq_geo2 = nq_dict[nq_dict.Variable==geo2_nq]
nq_geo2.columns = ["Variable",geo2_nq,geo2_nq+"_name"]
nq_geo2 = nq_geo2[[geo2_nq,geo2_nq+"_name"]]
# nq_geo1 = pd.read_excel("../Data/AR/panel/netquest_codebook_AR_provincia.xlsx")

# nq_geo2 = pd.read_excel("../Data/AR/panel/netquest_codebook_AR_departamento.xlsx")

Setting up the unique DF for geographies for IPUMS

In [9]:
ipums_geodf = census[[geo1_ipums,geo2_ipums]]
ipums_geodf.columns = ['geo1_code','geo2_code']
ipums_geodf.drop_duplicates(subset=['geo1_code','geo2_code'],inplace=True)

ipums_geodf['geo1_name'] = ipums_geodf.merge(ipums_geo1, 
                                             how='left', 
                                             left_on = "geo1_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

ipums_geodf['geo2_name'] = ipums_geodf.merge(ipums_geo2, 
                                             how='left', 
                                             left_on = "geo2_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

We have to get the melting action going here.

In [10]:
ipums_geodf = ipums_geodf.geo2_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo2_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo2_name')], value_name = "geo2_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo2_name'])
ipums_geodf.geo2_name = ipums_geodf.geo2_name.str.strip()

In [11]:
ipums_geodf = ipums_geodf.geo1_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo1_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo1_name')], value_name = "geo1_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo1_name'])
ipums_geodf.geo1_name = ipums_geodf.geo1_name.str.strip()

In [12]:
ipums_geodf.shape

(525, 4)

Now to Netquest--first, get the geo1- and geo2_name in there.

In [13]:
# province_labels = pd.read_excel("../Data/CL/provincia_dict.xlsx", encoding='Latin1', header=None)
nq_geodf = netquest.merge(nq_geo2, on=geo2_nq, how='left')\
                   .merge(nq_geo1, on=geo1_nq, how='left')[[geo1_nq,geo1_nq+'_name',geo2_nq,geo2_nq+'_name']]
nq_geodf.columns = ['geo1_code','geo1_name','geo2_code','geo2_name']
nq_geodf.drop_duplicates(inplace=True)

nq_geodf = nq_geodf[(nq_geodf['geo1_code'].notna())&(nq_geodf['geo2_code'].notna())]

In [367]:
# nq_geodf[nq_geodf.duplicated('geo2_code', keep=False)].sort_values('geo2_code')

Finding the duplicates in nq_geodf.geo2_code, and keeping the combinations that occur most often (which are quite clearly the good ones).

In [14]:
# nq_geodf['count'] = nq_geodf.apply(
#     lambda r: sum((netquest[geo1_nq]==r['geo1_code'])&
#                   (netquest[geo2_nq]==r['geo2_code'])),
#                                    axis=1)
# # nq_geodf.sort_values(['geo2_code','count']) \
# #     [(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep=False))]
# nq_geodf = nq_geodf.sort_values(['geo2_code','count']) \
#     [~(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep='last'))]

In [15]:
nq_geodf.shape

(811, 4)

In [16]:
ipums_geodf[ipums_geodf.geo2_name.str.contains("Pueyrredón")]

,geo1_code,geo2_code,geo2_name,geo1_name
61,6,6045,General Pueyrredón,Buenos Aires province


Next, a function for fuzzy-joining columns from dataframes

In [17]:
def fuzzy_join(df1, df2, varname, subset=False, cutoff=90):
    '''Returns a dataframe the length of df1, with the matches on the given variable
    
        Assumes the var is named the same in both dataframes.
        
        subset is a tuple or list of length 2 whose first element is a string indicating which
        variable in df1 has to be equal to which variable in df2 (specified by 2nd element)
        '''
    #figure out if there is a constraint
    if not subset:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)
        
    else:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)    

    
    # different match cases
    #morRatioMatch = ratio_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    oneRatioMatch = ratio_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noRatioMatch = ratio_matches.apply(lambda l: (l[0][1]<cutoff))

    #morPartiMatch = parti_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    onePartiMatch = parti_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noPartiMatch = parti_matches.apply(lambda l: (l[0][1]<cutoff))
    
    # pick out what's better
    matches = pd.Series([(np.nan,np.nan,np.nan)]*len(df1),index=df1.index)
    matches.loc[oneRatioMatch] = ratio_matches.loc[oneRatioMatch].apply(lambda l: l[0])
    matches.loc[(noRatioMatch&onePartiMatch)] = parti_matches.loc[(noRatioMatch&onePartiMatch)]\
        .apply(lambda l: l[0])
    matches.loc[(noRatioMatch&noPartiMatch)] = parti_matches.loc[(noRatioMatch&noPartiMatch)]\
        .apply(lambda l: l[0])
    
    df = pd.DataFrame()
    df['name'] = matches.apply(lambda l: l[0])
    df['score'] = matches.apply(lambda l: l[1])
    df['index'] = matches.apply(lambda l: l[2])
    df['parti_matches'] = parti_matches
    df['ratio_matches'] = ratio_matches
    
    return df

Use the new tool!

Manual fixes, as found necessary in the next step...

In [18]:
nq_geodf.loc[nq_geodf.geo1_name=="Ciudad Autónoma de Buenos Aires",
             'geo1_name'] = 'CABA'

ipums_geodf.loc[ipums_geodf.geo1_name=="City of Buenos Aires",
             'geo1_name'] = 'CABA'
nq_geodf.loc[(nq_geodf.geo1_name=="CABA"),'geo2_name'] = 'CABA'
ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA"),'geo2_name'] = 'CABA'

ipums_geodf.loc[ipums_geodf.geo1_name.str.contains("Buenos Aires province"),
                'geo1_name'] = "Buenos Aires"

ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Puan"),
                'geo2_name'] = "Puán"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("General San Martín"))\
                &(ipums_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "Ciudad Libertador San Martín"



ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Chascomus")),
                'geo2_name'] = "Chascomús"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Jose C. Paz")),
                'geo2_name'] = "José C. Paz"
# nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Adolfo Gonzales Chaves")),
#                 'geo2_name'] = "Adolfo González Chaves"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Paso de Indios"))\
                &(nq_geodf.geo1_name=="Chubut"),
                'geo2_name'] = "Paso de los Indios"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Coronel de Marina Leonardo Rosales"))\
                &(nq_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "Coronel de Marine L. Rosales"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Veinticinco de Mayo"))\
                &(ipums_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "25 de Mayo"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Pueyrredón"))\
                &(nq_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "General Pueyrredón"

We see that a lot of depa's are listed under various provinces. Can we utilize subpar matching later to get just the ones that are good?

The CABA districts require special attention. They are labelled school districts, but they are actually _comunas_.

The Barrios aren't provided for much of Netquest, so I'm just simplifying to the whole city here.

In [19]:

# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XIII')),
#              'geo2_name'] = 'Comuna 13'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('VIII')),
#              'geo2_name'] = 'Comuna 8'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XIV')),
#              'geo2_name'] = 'Comuna 14'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XII')),
#              'geo2_name'] = 'Comuna 12'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('VII')),
#              'geo2_name'] = 'Comuna 7'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('III')),
#              'geo2_name'] = 'Comuna 3'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XV')),
#              'geo2_name'] = 'Comuna 15'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('XI')),
#              'geo2_name'] = 'Comuna 11'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('IX')),
#              'geo2_name'] = 'Comuna 9'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('VI')),
#              'geo2_name'] = 'Comuna 6'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('IV')),
#              'geo2_name'] = 'Comuna 4'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('II')),
#              'geo2_name'] = 'Comuna 2'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('X')),
#              'geo2_name'] = 'Comuna 10'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('V')),
#              'geo2_name'] = 'Comuna 5'
# ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA")&\
#                 (ipums_geodf.geo2_name.str.contains('I')),
#              'geo2_name'] = 'Comuna 1'

In [20]:
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Puerto Madero','San Nicolás','Retiro','San Telmo','Constitución'])),
#             'geo2_name'] = 'Comuna 1'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Recoleta'])),
#             'geo2_name'] = 'Comuna 2'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Balvanera','San Cristóbal'])),
#             'geo2_name'] = 'Comuna 3'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['La Boca','Barracas', 'Parque Patricios', 'Nueva Pompeya'])),
#             'geo2_name'] = 'Comuna 4'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Almagro','Boedo'])),
#             'geo2_name'] = 'Comuna 5'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Caballito'])),
#             'geo2_name'] = 'Comuna 6'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Flores','Parque Chacabuco'])),
#             'geo2_name'] = 'Comuna 7'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Soldati', 'Villa Lugano', 'Villa Riachuelo'])),
#             'geo2_name'] = 'Comuna 8'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Parque Avellaneda', 'Mataderos', 'Liniers'])),
#             'geo2_name'] = 'Comuna 10'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Luro', 'Vélez Sársfield', 'Floresta', 'Monte Castro', 'Villa Real', 'Versalles'])),
#             'geo2_name'] = 'Comuna 10'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Devoto', 'Villa del Parque', 'Villa Santa Rita', 'Villa General Mitre'])),
#             'geo2_name'] = 'Comuna 11'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Pueyrredón', 'Villa Urquiza', 'Coghlan', 'Saavedra'])),
#             'geo2_name'] = 'Comuna 12'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Núñez', 'Belgrano', 'Colegiales'])),
#             'geo2_name'] = 'Comuna 13'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Palermo'])),
#             'geo2_name'] = 'Comuna 14'
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isin(
#     ['Villa Ortúzar', 'Chacarita', 'Villa Crespo', 'La Paternal', 'Agronomía', 'Parque Chas'])),
#             'geo2_name'] = 'Comuna 15'

## Fuzzy joining

In [21]:
nq_geodf[['geo1_match_name','geo1_match_score','geo1_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo1_name')[['name','score','index']]

In [22]:
nq_geodf[['geo2_match_name','geo2_match_score','geo2_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo2_name', 
             subset=['geo1_match_name','geo1_name']
            )[['name','score','index']]

## De-duping

Checking how good this worked.

In [23]:
# dupes = nq_geodf[nq_geodf.duplicated('geo2_code')].geo2_code

tmp = nq_geodf.copy()

# for code in dupes: 
#     if 'CABA' in tmp.loc[tmp.geo2_code==code].geo1_match_name.unique():
#         if int(tmp.loc[((tmp.geo2_code==code) & (tmp.geo1_name=="Buenos Aires")), 'geo2_match_score'])==100:
#             tmp = tmp[~((tmp.geo2_code==code) & (tmp.geo1_name=="CABA"))]
#     # if one of the geo2_match_names is an exact match, throw out any others
#     if sum(tmp.loc[tmp.geo2_code==code,"geo2_name"]==tmp.loc[tmp.geo2_code==code,"geo2_match_name"])>0:
#         tmp = tmp[~((tmp.geo2_code==code) & (tmp.geo2_name!=tmp.geo2_match_name))]

**What is still an issue??**

In [24]:
tmp[tmp.duplicated('geo2_code', keep=False)].sort_values('geo2_code')\
[["geo2_code","geo1_name","geo1_match_name","geo2_name","geo2_match_name","geo2_match_score","geo2_match_index"]]

,geo2_code,geo1_name,geo1_match_name,geo2_name,geo2_match_name,geo2_match_score,geo2_match_index
2280,1.0,Misiones,Misiones,25 de Mayo,25 de Mayo,100,252
132,1.0,Buenos Aires,Buenos Aires,25 de Mayo,25 de Mayo,100,446
217,1.0,Mendoza,Mendoza,25 de Mayo,Luján de Cuyo,60,228
1515,1.0,La Pampa,La Pampa,25 de Mayo,Toay,50,468
3470,2.0,Misiones,Misiones,9 de Julio,25 de Mayo,53,252
82,2.0,Buenos Aires,Buenos Aires,9 de Julio,9 de Julio,100,87
453,3.0,Catamarca,Catamarca,Adolfo Alsina,El Alto,57,374
37378,3.0,Corrientes,Corrientes,Adolfo Alsina,Esquina,57,149
43,3.0,Tucumán,Tucumán,Adolfo Alsina,Leales,50,339
2460,3.0,Buenos Aires,Buenos Aires,Adolfo Alsina,Adolfo Alsina,100,15


Success! This can be the geo-df.

In [25]:
nq_geodf = tmp

In [33]:
nq_geodf[(nq_geodf.geo2_match_score<71)][['geo1_name',
                                          'geo1_code',
#                                         'geo1_match_name',
                                        'geo2_name',
                                        'geo2_code',
                                        'geo2_match_name',
                                        'geo2_match_score'
                                       ]
                                      ].sort_values('geo1_name'
                                      )#.to_csv("./faulty_matches.csv")

,geo1_name,geo1_code,geo2_name,geo2_code,geo2_match_name,geo2_match_score
1185,Buenos Aires,2.0,Río Seco,150.0,Rojas,60
9,Buenos Aires,2.0,Marcos Paz,82.0,La Costa,62
10037,Catamarca,20.0,Magdalena,78.0,Andalgalá,56
5262,Catamarca,20.0,Navarro,89.0,Santa María,57
87153,Catamarca,20.0,San Andrés de Giles,112.0,Santa Rosa,60
18154,Catamarca,20.0,General Las Heras,51.0,La Paz,50
453,Catamarca,20.0,Adolfo Alsina,3.0,El Alto,57
54460,Catamarca,20.0,Maipú,79.0,Ambato,40
1434,Catamarca,20.0,Rojas,106.0,Tinogasta,60
53839,Catamarca,20.0,Alberti,5.0,Fray Mamerto Esquiú,57


In [386]:
has_ipums_geo = nq_geodf.geo2_match_score>70 #&nq_geodf.geo1_match_index.notna()

In [388]:
sum(has_ipums_geo)

206

In [285]:
nq_geodf['IPUMS_geo1_code'] = np.nan

nq_geodf.loc[has_ipums_geo,'IPUMS_geo1_code'] = nq_geodf[has_ipums_geo]\
                            .geo1_match_index\
                            .astype(int)\
                            .apply(
                                lambda i: ipums_geodf.loc[i,'geo1_code']
                            .astype(int)
)

We want to replace each nq_geo1 with the _code_ for the corresponding census geography.



Read in the municipio-centroids, and hopefully attach them here.

In [286]:
geo1_centroids = pd.read_csv('../Data/'+country+'/geography/'+country+'_geo1_centroids.csv', encoding='latin1')

In [287]:
nq_geodf_merged = nq_geodf.merge(geo1_centroids[['ADMIN_NAME','Y','X','IPUM'+year]], 
               left_on='IPUMS_geo1_code',
               right_on="IPUM"+year,
               how='left'
              ).drop('IPUM'+year, axis=1)

In [289]:
panel_geo = netquest.merge(nq_geodf_merged[['X','Y','geo1_code']],
               left_on=geo1_nq,
               right_on='geo1_code',
               how='left'
              )

In [290]:
panel_geo.shape[0]==netquest.shape[0]

True

In [291]:
panel_geo.to_csv('../Data/'+country+'/panel/'+country+'_netquest-panel_geo.csv')

And the same for census: attach geographies

In [292]:
census_geo = census.merge(geo1_centroids[['ADMIN_NAME','X',"Y",'IPUM'+year]],
                          left_on = geo1_ipums,
                          right_on='IPUM'+year,
                          how='left'
                         ).drop('IPUM'+year,axis=1)

In [293]:
census_geo.shape[0]==census.shape[0]

True

In [294]:
census_geo.to_csv('../Data/'+country+'/sample/'+country+'_ipums-census_geo.csv')